In [1038]:
import fiona
import pandas as pd
import geopandas
import os

In [1039]:
os.chdir('/Users/dannyha/SoCalEVCSNetwork')
os.getcwd()

'/Users/dannyha/SoCalEVCSNetwork'

# Data Importing and Cleaning

## SoCal Zipcodes and Geometry

In [1040]:
socal_zipcodes = pd.read_csv (r'data/socal_zipcodes/socal_zip.csv')

In [1041]:
socal_zipcodes.head()

,Unnamed: 0,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,54,6,91504,691504,B5,G6350,S,12783282,29685,34.204568,-118.326365,N,"POLYGON ((-118.348583 34.208461, -118.348564 3..."
1,55,6,91606,691606,B5,G6350,S,8631179,25464,34.185767,-118.388324,N,"POLYGON ((-118.403066 34.19394, -118.402564 34..."
2,56,6,90640,690640,B5,G6350,S,21990769,104151,34.015444,-118.111012,N,"POLYGON ((-118.128344 33.985893, -118.127359 3..."
3,57,6,91770,691770,B5,G6350,S,16634980,138149,34.065493,-118.084529,N,"POLYGON ((-118.107291 34.066222, -118.107323 3..."
4,59,6,90290,690290,B5,G6350,S,51935407,24096,34.107728,-118.615981,N,"POLYGON ((-118.640641 34.125821, -118.640133 3..."


In [1042]:
socal_zipcodes = socal_zipcodes[['ZCTA5CE10','GEOID10','INTPTLAT10','INTPTLON10','geometry']]

In [1043]:
socal_zipcodes = socal_zipcodes.rename(columns={"ZCTA5CE10": "Zip Code", "GEOID10": "GEOID", "INTPTLAT10": "Latitude",
                              "INTPTLON10": "Longitude", "geometry": "Geometry"})

In [1044]:
socal_zipcodes.head()

,Zip Code,GEOID,Latitude,Longitude,Geometry
0,91504,691504,34.204568,-118.326365,"POLYGON ((-118.348583 34.208461, -118.348564 3..."
1,91606,691606,34.185767,-118.388324,"POLYGON ((-118.403066 34.19394, -118.402564 34..."
2,90640,690640,34.015444,-118.111012,"POLYGON ((-118.128344 33.985893, -118.127359 3..."
3,91770,691770,34.065493,-118.084529,"POLYGON ((-118.107291 34.066222, -118.107323 3..."
4,90290,690290,34.107728,-118.615981,"POLYGON ((-118.640641 34.125821, -118.640133 3..."


In [1045]:
socal_zipcodes['Zip Code'] = socal_zipcodes['Zip Code'].apply(str)

In [1046]:
type(socal_zipcodes['Zip Code'][0])

str

## DMV Data

In [1047]:
vehicles = pd.read_csv (r'data/vehicle-count-as-of-1-1-2020.csv',header=1)
#vehicles = vehicles[['Zip Code','Model Year','Fuel','Make','Duty','Vehicles']]
vehicles = vehicles.drop(columns=['Date'])
vehicles.head()

/Users/dannyha/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,90001,2007,Gasoline,ACURA,Light,15
1,90002,2007,Gasoline,ACURA,Light,20
2,90003,2007,Gasoline,ACURA,Light,29
3,90004,2007,Gasoline,ACURA,Light,19
4,90006,2007,Gasoline,ACURA,Light,15


In [1048]:
#filtering by socal zipcodes
socal_vehicles = vehicles[vehicles['Zip Code'].isin(socal_zipcodes['Zip Code'])]

In [1049]:
#filtering light duty vehicles
socal_vehicles = socal_vehicles[socal_vehicles['Duty']=='Light']

In [1050]:
#filtering EVs
socal_evs = socal_vehicles[socal_vehicles['Fuel'].isin(['Battery Electric','Plug-in Hybrid'])]
socal_evs = socal_evs.reset_index()
socal_evs = socal_evs[['Zip Code','Model Year','Fuel','Make','Duty','Vehicles']]

In [1051]:
socal_evs.head()

,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,90013,2008,Battery Electric,OTHER/UNK,Light,1
1,90023,2008,Battery Electric,OTHER/UNK,Light,1
2,90024,2008,Battery Electric,OTHER/UNK,Light,3
3,90027,2008,Battery Electric,OTHER/UNK,Light,1
4,90049,2008,Battery Electric,OTHER/UNK,Light,3


In [1052]:
socal_evs['Vehicles'] = pd.to_numeric(socal_evs['Vehicles'])

In [1053]:
socal_vehicles_by_zip = socal_evs.groupby(['Zip Code']).sum('Vehicles')

In [1054]:
socal_vehicles_by_zip['Electric Vehicles'] = socal_vehicles_by_zip['Vehicles']
socal_vehicles_by_zip = socal_vehicles_by_zip.drop(columns='Vehicles')
socal_vehicles_by_zip.head()

,Electric Vehicles
Zip Code,
90001,134
90002,162
90003,136
90004,1294
90005,560


In [1055]:
vehicles = vehicles.drop([602395])

In [1056]:
vehicles['Vehicles'] = pd.to_numeric(vehicles['Vehicles'])
vehicles_by_zip = vehicles.groupby(['Zip Code']).sum('Vehicles')
vehicles_by_zip['Total Vehicles'] = vehicles_by_zip['Vehicles']
vehicles_by_zip = vehicles_by_zip.drop(columns = ['Vehicles'])

In [1057]:
ev_percentage = socal_vehicles_by_zip.merge(vehicles_by_zip, how='inner', on='Zip Code')
ev_percentage

,Electric Vehicles,Total Vehicles
Zip Code,,
90001,134,69216.0
90002,162,62696.0
90003,136,83462.0
90004,1294,66876.0
90005,560,34936.0
...,...,...
93543,60,22758.0
93550,386,108116.0
93551,1074,89352.0


In [1058]:
ev_percentage['EV Percentage'] = ev_percentage['Electric Vehicles'] / ev_percentage['Total Vehicles']
ev_percentage = ev_percentage[['Electric Vehicles','Total Vehicles','EV Percentage']]
ev_percentage.head()

,Electric Vehicles,Total Vehicles,EV Percentage
Zip Code,,,
90001,134,69216.0,0.001936
90002,162,62696.0,0.002584
90003,136,83462.0,0.001629
90004,1294,66876.0,0.019349
90005,560,34936.0,0.016029


In [1059]:
ev_percentage.to_excel("ev_percentage.xlsx") 

## LEHD Data

In [1060]:
commuting = pd.read_csv (r'data/commuting/ca_od_main_JT00_2018.csv')

In [1061]:
commuting.columns = ['Workplace Census Block Code', 'Residence Census Block Code',
                    'Total number of jobs', 'Number of jobs of workers age 29 or younger',
                    'Number of jobs for workers age 30 to 54', 'Number of jobs for workers age 55 or older',
                    'Number of jobs with earnings $1250/month or less',
                     'Number of jobs with earnings $1251/month to $3333/month',
                    'Number of jobs with earnings greater than $3333/month',
                    'SI01','SI02','SI01','date']

In [1062]:
commuting_filtered = commuting[['Workplace Census Block Code', 'Residence Census Block Code',
                    'Total number of jobs']]
commuting_filtered = commuting_filtered.reset_index().drop(columns=['index'])

In [1063]:
tract_to_zip = pd.read_csv (r'data/TRACT_ZIP_032021.csv')
tract_to_zip

,TRACT,ZIP,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1001020100,36067,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
1,1001020200,36067,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
2,1001020300,36067,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
3,1001020400,36067,PRATTVILLE,AL,0.024142,0.583851,0.25,0.070064
4,1001020400,36066,PRATTVILLE,AL,0.975858,0.416149,0.75,0.929936
...,...,...,...,...,...,...,...,...
172033,78030960800,802,ST THOMAS,VI,1.000000,1.000000,1.00,1.000000
172034,78030960900,802,ST THOMAS,VI,0.000000,0.000000,1.00,1.000000
172035,78030961000,802,ST THOMAS,VI,1.000000,1.000000,1.00,1.000000
172036,78030961100,802,ST THOMAS,VI,1.000000,1.000000,1.00,1.000000


In [1064]:
tract_to_zip['ZIP'] = tract_to_zip['ZIP'].apply(str)

In [1065]:
socal_zips = tract_to_zip[tract_to_zip['ZIP'].isin(socal_zipcodes['Zip Code'])]

In [1066]:
socal_zips.head()

,TRACT,ZIP,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
10911,6037101110,91042,TUJUNGA,CA,1.000000,1.000000,1.000000,1.000000
10912,6037101122,91042,TUJUNGA,CA,1.000000,1.000000,0.000000,1.000000
10913,6037101210,91042,TUJUNGA,CA,0.780202,0.628472,0.993399,0.785909
10915,6037101220,91042,TUJUNGA,CA,1.000000,1.000000,1.000000,1.000000
10916,6037101300,91214,LA CRESCENTA,CA,0.098039,0.043011,0.166667,0.095492


In [1067]:
socal_zips['TRACT'] = socal_zips['TRACT'].apply(int)

<ipython-input-1067-a85bbcb9a734>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socal_zips['TRACT'] = socal_zips['TRACT'].apply(int)


In [1068]:
#RES_RATIO = The ratio of residential addresses in the ZIP – Tract, County, or CBSA part 
#to the total number of residential addresses in the entire ZIP.

socal_zips_cleaned = socal_zips[['TRACT','ZIP','RES_RATIO']]

In [1069]:
#converting zip_and_tract to dictionary
zip_and_tract_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')

<ipython-input-1069-23ca5f4ecff8>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  zip_and_tract_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')


In [1070]:
#remove last four digit of block code to make is census tract code
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].apply(str)
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].apply(str)
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].str.replace(r'\d{4}$', '')
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].str.replace(r'\d{4}$', '')

In [1071]:
commuting_filtered.columns = ['Workplace Census Tract', 'Residence Census Tract','Total Jobs']

In [1072]:
#changing Total Jobs column to integers
commuting_filtered['Total Jobs'] = commuting_filtered['Total Jobs'].apply(int)

In [1073]:
# removing insignificant rows
commuting_filtered = commuting_filtered[commuting_filtered['Total Jobs'] > 10]

In [1074]:
socal_zips_cleaned['TRACT'] = socal_zips_cleaned['TRACT'].apply(int)

<ipython-input-1074-39e02b299d53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socal_zips_cleaned['TRACT'] = socal_zips_cleaned['TRACT'].apply(int)


In [1075]:
commuting_filtered['TRACT'] = commuting_filtered['Residence Census Tract']

In [1079]:
commuting_filtered.head()

,Workplace Census Tract,Residence Census Tract,Total Jobs,TRACT
94464,6001403400,6001430101,11,6001430101
94821,6001403400,6001433103,11,6001433103
107962,6001404000,6001425102,11,6001425102
152685,6001408000,6001408000,12,6001408000
220371,6001422300,6001420300,27,6001420300


In [1081]:
socal_zips_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')
socal_zips_dict = {int(key):socal_zips_dict[key] for key in socal_zips_dict}

<ipython-input-1081-0094f37d3faa>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  socal_zips_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')


In [1082]:
commuting_filtered['Workplace Census Tract'] = commuting_filtered['Workplace Census Tract'].apply(int)

In [1083]:
commuting_grouped = commuting_filtered.groupby(
    ['Residence Census Tract','Workplace Census Tract']).sum('Total Jobs')
commuting_grouped

Total Jobs
Residence Census Tract Workplace Census Tract            
6001408000             6001408000                      12
6001420300             6001422300                      27
                       6001422600                      45
                       6075017700                      28
6001420400             6001422300                      14
...                                                   ...
6113011206             6095253300                     109
                       6113010800                      12
                       6113011500                      39
6113011300             6095253300                      19
                       6113011300                      37

[2787 rows x 1 columns]

In [1085]:
commuting_grouped.sort_values('Total Jobs',ascending=False).head(20)

,,Total Jobs
Residence Census Tract,Workplace Census Tract,
6059062614,6059062614,1355
6053014105,6053014104,374
6071007302,6071007303,369
6081610202,6081611700,355
6083002930,6083002922,343
6073008363,6073008305,317
6051000200,6051000200,271
6075061500,6075061500,263
6085505008,6085505008,263


In [720]:
ev_geo = zipcodes.merge(ev_percentage,on='Zip Code')

In [723]:
ev_geo['ev_percentage'] = ev_geo['Electric Vehicles'] / ev_geo['Total Vehicles']

In [726]:
ev_geo

,Zip Code,geometry,index,Electric Vehicles,Total Vehicles,ev_percentage
0,91504,"POLYGON ((-118.34858 34.20846, -118.34856 34.2...",236,894,42066.0,0.021252
1,91606,"POLYGON ((-118.40307 34.19394, -118.40256 34.1...",243,736,61560.0,0.011956
2,90640,"POLYGON ((-118.12834 33.98589, -118.12736 33.9...",129,842,98642.0,0.008536
3,91770,"POLYGON ((-118.10729 34.06622, -118.10732 34.0...",261,1484,101572.0,0.014610
4,90290,"POLYGON ((-118.64064 34.12582, -118.64013 34.1...",97,710,11192.0,0.063438
...,...,...,...,...,...,...
346,92694,"POLYGON ((-117.65254 33.56291, -117.65168 33.5...",309,3014,48286.0,0.062420
347,92831,"POLYGON ((-117.91359 33.86940, -117.91359 33.8...",328,1138,49684.0,0.022905
348,92707,"MULTIPOLYGON (((-117.87814 33.66436, -117.8780...",315,612,87650.0,0.006982
349,92648,"MULTIPOLYGON (((-118.01199 33.69955, -118.0120...",290,2830,80342.0,0.035224


In [727]:
ev_geo.to_file("socal_ev.shp")

<ipython-input-727-e99afb744cc3>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ev_geo.to_file("socal_ev.shp")
